In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

import time
import pandas as pd
from pathlib import Path

# Chrome WebDriver 옵션 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # 브라우저 GUI 비활성화
chrome_options.add_argument('--no-sandbox')  # 샌드박스 옵션 비활성화
chrome_options.add_argument('--disable-dev-shm-usage')  # 리소스 제한 방지
chrome_options.add_argument('lang=ko_KR')  # 브라우저 언어 설정

# 웹 크롤링 시작
browser = webdriver.Chrome(options=chrome_options)
browser.get('https://www.starbucks.co.kr/store/store_map.do?disp=locale')
browser.implicitly_wait(5)

columns = ['매장명', '위도', '경도', '매장타입', '주소']
all_starbucks_list = []  # 모든 매장 데이터를 저장할 리스트

# 매장 정보 크롤링 함수
def crawl_starbucks_by_region(region_index):
    try:
        loc_search_button = browser.find_element(By.XPATH, '//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/header[2]/h3/a')
        browser.execute_script("arguments[0].click();", loc_search_button)
        time.sleep(5)

        region_button = browser.find_element(By.XPATH, f'//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[{region_index}]/a')
        region_name = region_button.text
        print(f"{region_index}번째 지역: {region_name}")
        region_button.click()
        time.sleep(2)

        if region_index < 17:
            all_button = browser.find_element(By.XPATH, '//*[@id="mCSB_2_container"]/ul/li[1]/a')
            all_button.click()
            time.sleep(5)

        stores = browser.find_elements(By.CLASS_NAME, 'quickResultLstCon')
        print(f"{region_name} 매장 수: {len(stores)}")

        region_stores = []
        for store in stores:
            name = store.get_attribute('data-name')
            lat = store.get_attribute('data-lat')
            lng = store.get_attribute('data-long')
            store_type = "리저브" if name.endswith('R') else ("드라이브스루" if name.endswith('DT') else "일반")
            address = str(store.find_element(By.CLASS_NAME, 'result_details').get_attribute('outerHTML')).split('>')[1].split('<br')[0].strip()
            region_stores.append([name, lat, lng, store_type, address])

        df = pd.DataFrame(region_stores, columns=columns)
        
        base_dir = Path.cwd().parents[5]
        csv_path = base_dir / 'resources' / 'data' / f'starbucks_store_list_{region_name}.csv'
        
        df.to_csv(csv_path, index=False)
        print(f"{region_name} 저장 완료")

        return region_stores

    except Exception as e:
        print(f"오류 발생: {e}")
        return []

# 모든 지역 크롤링
for i in range(1, 18):
    print(f"----- {i}번째 지역 크롤링 시작 -----")
    region_stores = crawl_starbucks_by_region(i)
    all_starbucks_list.extend(region_stores)
    print(f"----- {i}번째 지역 크롤링 완료 -----")

# 전체 데이터 저장
all_starbucks_df = pd.DataFrame(all_starbucks_list, columns=columns)
base_dir = Path.cwd().parents[5]
csv_path = base_dir / 'resources' / 'data' / 'all_starbucks_store_list.csv'
all_starbucks_df.to_csv(csv_path, index=False)
print("크롤링 완료. 모든 데이터 저장.")
browser.quit()

----- 1번째 지역 크롤링 시작 -----
1번째 지역: 서울
서울 매장 수: 642
서울 저장 완료
----- 1번째 지역 크롤링 완료 -----
----- 2번째 지역 크롤링 시작 -----
2번째 지역: 경기
경기 매장 수: 501
경기 저장 완료
----- 2번째 지역 크롤링 완료 -----
----- 3번째 지역 크롤링 시작 -----
3번째 지역: 광주
광주 매장 수: 68
광주 저장 완료
----- 3번째 지역 크롤링 완료 -----
----- 4번째 지역 크롤링 시작 -----
4번째 지역: 대구
대구 매장 수: 93
대구 저장 완료
----- 4번째 지역 크롤링 완료 -----
----- 5번째 지역 크롤링 시작 -----
5번째 지역: 대전
대전 매장 수: 71
대전 저장 완료
----- 5번째 지역 크롤링 완료 -----
----- 6번째 지역 크롤링 시작 -----
6번째 지역: 부산
부산 매장 수: 146
부산 저장 완료
----- 6번째 지역 크롤링 완료 -----
----- 7번째 지역 크롤링 시작 -----
7번째 지역: 울산
울산 매장 수: 34
울산 저장 완료
----- 7번째 지역 크롤링 완료 -----
----- 8번째 지역 크롤링 시작 -----
8번째 지역: 인천
인천 매장 수: 82
인천 저장 완료
----- 8번째 지역 크롤링 완료 -----
----- 9번째 지역 크롤링 시작 -----
9번째 지역: 강원
강원 매장 수: 43
강원 저장 완료
----- 9번째 지역 크롤링 완료 -----
----- 10번째 지역 크롤링 시작 -----
10번째 지역: 경남
경남 매장 수: 80
경남 저장 완료
----- 10번째 지역 크롤링 완료 -----
----- 11번째 지역 크롤링 시작 -----
11번째 지역: 경북
경북 매장 수: 62
경북 저장 완료
----- 11번째 지역 크롤링 완료 -----
----- 12번째 지역 크롤링 시작 -----
12번째 지역: 전남
전남 매장 수: 35
전남 저장 완료
----- 1